Celem laboratorium było wykonanie implementacji algorytmu Apriori oraz przetestowanie go na dwóch zbiorach danych.

Importy:

In [1]:
from __future__ import annotations
import time
import pandas as pd
import copy
from itertools import combinations_with_replacement

Pomocnicza klasa realizująca wypisywanie postępów na ekran oraz informująca o czasie wykonania danej funkcji:

In [2]:
class VerbosityHelper:
    def __init__(self, is_enabled: bool = True, verbosity_level: int = 1):
        self.is_enabled: bool = is_enabled
        self.verbosity_level: int = verbosity_level

    def verbose(self, callback: callable, params: list = [], required_level: int = 1, message: str = 'Time of execution'):
        if self.is_enabled is False or self.verbosity_level < required_level:
            return callback(*params)

        t1 = time.time()
        result = callback(*params)
        t2 = time.time()

        time_result = t2 - t1
        indents = ''.join(['\t' for _ in range(required_level)])
        print(f'{indents}{message} - {time_result:.4f}s')

        return result

Klasa algorytmu Apriori. 
Jedna publiczna metoda `generate_rules` odpowiada za generowanie reguł i wykorzystuje ona dwie pomocnicze chronione metody `_generate_candidates` oraz `_create_rules`.

In [3]:
class Apriori:
    def __init__(self, min_supp: float = 0.3, min_conf: float = 0.7, verbose: bool = True, verbosity_level: int = 1):
        self.min_supp: float = min_supp
        self.min_conf: float = min_conf

        self.frequents: list = []
        self.rules: pd.DataFrame = pd.DataFrame()

        self.__verbosity_helper = VerbosityHelper(verbose, verbosity_level)
        self.__dataset_length: int = 0

    def generate_rules(self, dataset: pd.DataFrame) -> pd.DataFrame:
        self.__dataset_length = dataset.shape[0]

        self.frequents = self.__verbosity_helper.verbose(self._generate_candidates, [dataset], message='Time of generating candidates')
        self.rules = self.__verbosity_helper.verbose(self._create_rules, message='Time of creating rules')

        return self.rules

    def _generate_candidates(self, dataset: pd.DataFrame) -> list:
        frequents: list = []
        f: dict = self.__verbosity_helper.verbose(self.__get_unique_items_from_dataset, [dataset], 2, 'Time of selecting uniques items from dataset')

        while len(f) != 0:
            frequents.append(copy.deepcopy(f))

            c = self.__generate_candidates_from_frequencies(f, dataset)
            c = self.__prune_candidates(c)

            f = copy.deepcopy(c)

        return frequents

    def _create_rules(self) -> pd.DataFrame:
        frequents_mapped = list(reversed([{key: set() for key in frequencies} for frequencies in self.frequents[1:]]))
        generated_rules = pd.DataFrame(columns=['rule', 'supp', 'conf', 'lift'])

        for frequencies in frequents_mapped:
            for keys in frequencies.keys():
                df = pd.DataFrame(columns=['rule', 'supp', 'conf', 'lift'])
                generated_rules = pd.concat([generated_rules, self.__generate_rules_helper(keys, df)],
                                            ignore_index=True)

        return generated_rules.sort_values(by=['lift'], ascending=False)

    @staticmethod
    def __get_unique_items_from_dataset(dataset: pd.DataFrame, with_counts: bool = True) -> dict | list:
        uniques = {}

        for transaction_iter, transaction_items in dataset.iterrows():
            for value in transaction_items:
                if value is None or pd.isnull(value):
                    continue

                if value in uniques.keys():
                    uniques[value] += 1
                else:
                    uniques[value] = 1

        if not with_counts:
            return list(uniques.keys())

        return uniques

    @staticmethod
    def __generate_candidates_from_frequencies(frequencies: dict, dataset: pd.DataFrame) -> dict:
        """
        Generates all combinations of items in transactions dataset and count them.

        Example:
             {'a': 5, 'b': 7, 'c': 7}, {('a', 'b'): 3, ('b', 'c'): 5}, {('a', 'b', 'c'): 2}
        """
        candidates = {}
        first_item = list(frequencies.keys())[0]

        if type(first_item) == str:  # for one element item set
            candidates = {tuple(sorted([a, b])): 0 for (a, b) in combinations_with_replacement(list(frequencies.keys()), 2) if a != b}
        else:  # for multi element item set
            for frequency_item1 in frequencies.keys():
                frequency_item1_set = set(frequency_item1)

                for frequency_item2 in frequencies.keys():
                    frequency_item2_set = set(frequency_item2)

                    if len(frequency_item1_set.intersection(frequency_item2_set)) == 0 or frequency_item1_set == frequency_item2_set:
                        continue

                    key = tuple(sorted({*frequency_item1_set, *frequency_item2_set}))
                    candidates[key] = 0

        for keys in candidates.keys():
            where_is_keys = dataset.isin(keys).sum(axis=1) >= len(keys)
            candidates[keys] = where_is_keys.sum()

        return candidates

    def __prune_candidates(self, candidates: dict) -> dict:
        pruned_candidates = {}

        for key, count in candidates.items():
            if count / self.__dataset_length >= self.min_supp:
                pruned_candidates[key] = count

        return pruned_candidates

    def __generate_rules_helper(self, keys: list, local_rules: pd.DataFrame, local_word: list = []) -> pd.DataFrame:
        if len(keys) == 1:
            return local_rules

        for key_iter, key in enumerate(keys):
            keys_copy = copy.deepcopy(list(keys))  # copy to prevent modify original looped keys

            removed = keys_copy.pop(key_iter)
            keys_copy_tuple = tuple(keys_copy)  # tuple, because list is unhashable type

            # resulting element: from {} => {} the word is the second pair of brackets
            word = copy.deepcopy(local_word)
            word.append(removed)
            word = list(sorted(word))
            word_tuple = tuple(word)  # tuple, because list is unhashable type

            # calculating conf for pruning
            supp_expression = self.__find_expression_count_in_frequents(tuple(keys)) / self.__dataset_length
            supp_resulting = self.__find_expression_count_in_frequents(tuple([*word])) / self.__dataset_length
            conf = supp_expression / supp_resulting

            # reverse pruning - add only when condition is meet
            if conf >= self.min_conf:
                supp_factors = self.__find_expression_count_in_frequents(keys_copy_tuple) / self.__dataset_length

                local_rule = pd.Series({
                    'rule': f'{keys_copy_tuple} => {word_tuple}',
                    'supp': {'expr': supp_expression, 'factors': supp_factors, 'resulting': supp_resulting},
                    'conf': conf})
                local_rules = pd.concat([local_rules, local_rule.to_frame().T], ignore_index=True)

            generated_rules = self.__generate_rules_helper(keys_copy, local_rules, word)
            generated_rules = generated_rules.sort_values(by=['conf'], ascending=False).drop_duplicates(subset=['rule'])
            local_rules = pd.concat([local_rules, generated_rules], ignore_index=True)

        # removing duplicated rules and calc lift for rules
        local_rules = local_rules.sort_values(by=['conf'], ascending=False).drop_duplicates(subset=['rule'])
        local_rules['lift'] = self.__calc_lift_for_rules(local_rules)

        return local_rules

    def __find_expression_count_in_frequents(self, expression: tuple) -> int:
        expression = tuple(sorted(expression))
        expression = expression[0] if len(expression) == 1 else expression
        index_to_search = len(expression) - 1 if type(expression) != str else 0

        return self.frequents[index_to_search][expression]

    @staticmethod
    def __calc_lift_for_rules(local_rules: pd.DataFrame) -> list:
        lifts = []

        for rule_iter, rule in local_rules.iterrows():
            supp = rule['supp']

            lift = supp['expr'] / (supp['factors'] * supp['resulting'])
            lifts.append(lift)

        return lifts

Wykonanie algorytmu dla zbioru z punktu pierwszego instrukcji laboratoryjnej:

In [4]:
dataset = pd.DataFrame.from_dict({
        "t1": {'a', 'b', 'c'},
        "t2": {'b', 'c', 'd'},
        't3': {'a', 'b', 'd', 'e'},
        't4': {'a', 'c', 'd', 'e'},
        't5': {'b', 'c', 'd', 'e'},
        't6': {'b', 'd', 'c'},
        't7': {'c', 'd'},
        't8': {'a', 'b', 'c'},
        't9': {'a', 'd', 'e'},
        't10': {'b', 'd'},
    }, orient='index')

min_supp = 0.3
min_conf = 0.7

apriori = Apriori(min_supp, min_conf, verbose=True, verbosity_level=1)
print(apriori.generate_rules(dataset).to_markdown())

	Time of generating candidates - 0.0060s
	Time of creating rules - 0.0562s
|    | rule                 | supp                                            |     conf |     lift |
|---:|:---------------------|:------------------------------------------------|---------:|---------:|
|  3 | ('e',) => ('a', 'd') | {'expr': 0.4, 'factors': 0.4, 'resulting': 0.3} | 1.33333  | 3.33333  |
|  6 | ('a', 'd') => ('e',) | {'expr': 0.3, 'factors': 0.3, 'resulting': 0.4} | 0.75     | 2.5      |
|  4 | ('d',) => ('a', 'e') | {'expr': 0.4, 'factors': 0.8, 'resulting': 0.3} | 1.33333  | 1.66667  |
|  5 | ('a',) => ('d', 'e') | {'expr': 0.3, 'factors': 0.5, 'resulting': 0.4} | 0.75     | 1.5      |
| 11 | ('a',) => ('e',)     | {'expr': 0.3, 'factors': 0.5, 'resulting': 0.4} | 0.75     | 1.5      |
|  1 | ('c',) => ('b', 'd') | {'expr': 0.5, 'factors': 0.7, 'resulting': 0.5} | 1        | 1.42857  |
|  2 | ('b',) => ('c', 'd') | {'expr': 0.5, 'factors': 0.7, 'resulting': 0.5} | 1        | 1.42857  |
|  0 | 

Wykonanie algorytmu dla zbioru 'Store_data.csv':

In [5]:
store = pd.read_csv('Store_data.csv')

min_supp = 0.005
min_conf = 1.5

apriori = Apriori(min_supp, min_conf, verbose=True, verbosity_level=2)
print(apriori.generate_rules(store).to_markdown())

		Time of selecting uniques items from dataset - 0.1850s
	Time of generating candidates - 104.2419s
	Time of creating rules - 2.3391s
|     | rule                                                        | supp                                                                                              |    conf |     lift |
|----:|:------------------------------------------------------------|:--------------------------------------------------------------------------------------------------|--------:|---------:|
| 240 | ('ground beef',) => ('herb & pepper', 'spaghetti')          | {'expr': 0.0392, 'factors': 0.09826666666666667, 'resulting': 0.016266666666666665}               | 2.40984 | 24.5234  |
| 189 | ('ground beef',) => ('herb & pepper', 'mineral water')      | {'expr': 0.040933333333333335, 'factors': 0.09826666666666667, 'resulting': 0.017066666666666667} | 2.39844 | 24.4074  |
|   1 | ('spaghetti',) => ('chocolate', 'milk', 'mineral water')    | {'expr': 0.05973333333333333, 'f